In [7]:
#!/usr/bin/env python3
"""
Scan every .c / .cpp file in a directory tree with identify_cwe()
and write the results (including filename) to a CSV.
"""
import sys
import csv
import pathlib
from src.eval_demo import identify_cwe   # ← make sure detect_cwe.py is on PYTHONPATH

# ---------------------------------------------------------------------------
def scan_directory(src_dir: pathlib.Path) -> list[dict]:
    """Return a list of result-dicts for all C / C++ files under src_dir."""
    records: list[dict] = []
    patterns = ("*.c", "*.cpp", "*.cc", "*.cxx", "*.h", "*.hpp")
    files = [p for pat in patterns for p in src_dir.rglob(pat)]
    if not files:
        print(f"[WARN] No C/C++ files found in {src_dir}")
        return records

    print(f"Scanning {len(files)} files …")
    for path in files:
        code = path.read_text(encoding="utf-8", errors="ignore")
        res  = identify_cwe(code)
        res["file"] = str(path.relative_to(src_dir))  # nice relative path
        records.append(res)
    return records

# ---------------------------------------------------------------------------
def write_csv(records: list[dict], csv_path: pathlib.Path):
    if not records:
        print("Nothing to write (no records).")
        return
    fieldnames = ["file", "nearest_cwe", "is_vulnerable",
                  "severity", "vulnerable_code"]
    with csv_path.open("w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(records)
    print(f"✅  Results written → {csv_path}")

# ---------------------------------------------------------------------------
def main():
    src_dir  = pathlib.Path("../data/test")
    if not src_dir.is_dir():
        print(f"Error: {src_dir} is not a directory")
        sys.exit(1)

    csv_out  = pathlib.Path("../data/results/cwe_scan_results.csv")

    records  = scan_directory(src_dir)
    write_csv(records, csv_out)

main()


Scanning 11 files …
✅  Results written → ..\data\results\cwe_scan_results.csv
